In [1]:
import os
import cv2

# ディレクトリ設定
IMAGE_DIR = "images"
LABEL_DIR = "labels"
OUTPUT_DIR = "output_vis"  # 出力先ディレクトリ
IMAGE_SIZE = 512

# 出力先ディレクトリ作成
os.makedirs(OUTPUT_DIR, exist_ok=True)

# クラスごとの色指定
COLORS = {
    0: (255, 0, 0),    # Circle - Blue
    1: (0, 255, 0),    # Triangle - Green
    2: (0, 0, 255),    # Square - Red
}

# 対象画像一覧
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.endswith(".png")])

for img_file in image_files:
    img_path = os.path.join(IMAGE_DIR, img_file)
    label_path = os.path.join(LABEL_DIR, img_file.replace(".png", ".txt"))

    # 画像読み込み
    img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)

    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 6:
                    continue
                class_id, cx, cy, w, h, area_norm = parts
                class_id = int(class_id)
                cx = float(cx)
                cy = float(cy)
                w = float(w)
                h = float(h)
                area_norm = float(area_norm)

                # 非正規化のバウンディングボックス
                x1 = int((cx - w / 2) * IMAGE_SIZE)
                y1 = int((cy - h / 2) * IMAGE_SIZE)
                x2 = int((cx + w / 2) * IMAGE_SIZE)
                y2 = int((cy + h / 2) * IMAGE_SIZE)

                area_px = int(area_norm * IMAGE_SIZE * IMAGE_SIZE)
                color = COLORS.get(class_id, (255, 255, 255))

                # 図形に枠を描画
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                # クラスIDと面積（整数）を表示
                label_text = f"ID:{class_id} A:{area_px}"
                cv2.putText(img, label_text, (x1 + 2, y1 - 6),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # 保存
    save_path = os.path.join(OUTPUT_DIR, img_file)
    cv2.imwrite(save_path, img)

print(f"可視化画像を {OUTPUT_DIR} フォルダに保存しました。")


可視化画像を output_vis フォルダに保存しました。
